In [1]:
sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# Read Mongo collection in dataframe

In [10]:
#For 1000 documents : smallBrevet // For 10000 documents : brevets
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://127.0.0.1/local.smallBrevet").load()

In [11]:
type(df)

pyspark.sql.dataframe.DataFrame

In [12]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- description: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- publicationNumber: string (nullable = true)
 |-- title: string (nullable = true)



In [13]:
df.show()

+--------------------+--------------------+--------------------+-----------------+--------------------+
|                 _id|         description|            keywords|publicationNumber|               title|
+--------------------+--------------------+--------------------+-----------------+--------------------+
|[5e20910edc0c9c2b...|(EP3544210)|This ...|Digital communica...|    WO2018/113539|(EP3544210)|Clock...|
|[5e20910edc0c9c2b...|(EP3546818)|BACKG...|Electrical machin...|     MX2016015395|(EP3546818)|Optic...|
|[5e20910edc0c9c2b...|(EP3378433)|CROSS...|  Medical technology|    WO2017/160004|(EP3378433)|Guide...|
|[5e20910edc0c9c2b...|(EP2859171)|TECHN...|   Civil engineering|        CA2876375|(EP2859171)|Modul...|
|[5e20910edc0c9c2b...|(WO2015/000987)|M...|Biotechnology|Mea...|   DE102013011495|(EP3017292)|Metho...|
|[5e20910edc0c9c2b...|(EP3567461)|TECHN...|Basic communicati...|    WO2019/173934|(EP3567461)|Press...|
|[5e20910edc0c9c2b...|(EP3557422)|TECHN...| Computer technology|

In [ ]:
# Etude sur les keywords pour appréhender les outputs

In [95]:
dfKeywords = df.select('keywords').toPandas()

In [99]:
dfKeywords["keywords"]

0               Digital communication|Telecommunications
1      Electrical machinery, apparatus, energy|Optics...
2                                     Medical technology
3                                      Civil engineering
4                              Biotechnology|Measurement
5      Basic communication processes|Computer technol...
6                                    Computer technology
7               Digital communication|Telecommunications
8                Other consumer goods|Telecommunications
9      Computer technology|Control|Digital communication
10                                   Computer technology
11                             Basic materials chemistry
12                                    Telecommunications
13                                  Chemical engineering
14                                    Medical technology
15     Electrical machinery, apparatus, energy|Medica...
16              Digital communication|Telecommunications
17                    Medical t

# Nettoyage du texte round1 : rm, lower(), etc.

In [70]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\w*ϕ\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [71]:
#On doit pouvoir appliquer peut-être directement sur un RDD, a minima ne pas passer par panda
import pandas as pd

description = df.select('description').toPandas()
data_clean = description['description'].apply(round1)
data_clean

0       application claims priority to chinese patent...
1       field of the inventionthis invention is refer...
2       to related applicationthis application claims...
3       fieldthe present disclosure relates generally...
4       for determining the concentration of a substa...
5       fieldthe present application relates to the f...
6       fieldthe present application relates to the f...
7       application claims priority to chinese patent...
8       storage and access systemrelated applications...
9       control unitdescriptionthe present invention ...
10      and device for providing an imagetechnical fi...
11      compositions with enhanced cold properties an...
12      extenders control methods therefor and coaxia...
13      for mixingthe invention relates to apparatus ...
14      of the inventionthe present invention relates...
15      medical devices with electrically isolatedbat...
16      fieldthe present invention relates to the fie...
17      and methods for control

In [59]:
description['description']

0      (EP3544210)|This application claims priority t...
1      (EP3546818)|BACKGROUND.|1. Field of the invent...
2      (EP3378433)|CROSS-REFERENCE TO RELATED APPLICA...
3      (EP2859171)|TECHNICAL FIELD|The present disclo...
4      (WO2015/000987)|Method for determining the con...
5      (EP3567461)|TECHNICAL FIELD|The present applic...
6      (EP3557422)|TECHNICAL FIELD|The present applic...
7      (EP3541049)|This application claims priority t...
8      (WO2018/112503)|PHONE STORAGE AND ACCESS SYSTE...
9      (WO2018/109739)|TITLE|"FIRE-PREVENTION CONTROL...
10     (WO2018/097632)|METHOD AND DEVICE FOR PROVIDIN...
11     (WO2018/138412)|FUEL COMPOSITIONS WITH ENHANCE...
12     (WO2018/113590)|VIDEO EXTENDERS, CONTROL METHO...
13     (WO2018/115893)|Apparatus For Mixing|The inven...
14     (EP3258868)|FIELD OF THE INVENTION|The present...
15     (WO2016/077295)|IMPLANTABLE MEDICAL DEVICES WI...
16     (EP3086589)|TECHNICAL FIELD|The present invent...
17     (WO2015/089421)|DESCRIPT

In [72]:
# boucle pour comparer les data clean et pas clean au bout du round 1. 
#Resultats curieux : + de termes mais - de termes uniques 
i=0
compteur = 0
while i < len(description['description']):
    compteur = compteur + len(data_clean[i])
    i = i+1
    print(compteur)

65700
88115
106611
155586
185654
240949
271169
347017
361513
403656
456474
496254
590122
609033
652113
668273
790885
836272
875332
916846
939310
986827
1019732
1118727
1177021
1374246
1414387
1447720
1490910
1525084
1583591
1622851
1656938
1714064
1769620
1812322
1878747
1899352
2038021
2121230
2146195
2231806
2255212
2293610
2405303
2445322
2491629
2515593
2536227
2549935
2742954
2774547
2929489
3117801
3219459
3277469
3438499
3468295
3522567
3568556
3697761
3760519
3790118
3936084
3965224
4175897
4241864
4309806
4344097
4377243
4415742
4452752
4526889
4710077
4742170
4921629
5124185
5134086
5239002
5291105
5316482
5363844
5428586
5469867
5504605
5519813
5550643
5571004
5595523
5644816
5739393
5873575
5986158
6009277
6031397
6097518
6142737
6162396
6221939
6259909
6344136
6410738
6514885
6584394
6640921
6796076
6861984
6946301
7008307
7137619
7187031
7416428
7535460
7593566
7675548
7849435
7941381
7974690
7988146
8057971
8142120
8194048
8269901
8423792
8492702
8564247
8728442
8755614


71080422
71126316
71275024
71291800
71315709
71332060
71436431
71510747
71539657
71587294
71656972
71739624


In [61]:
len(data_clean[11])

39780

# CountVectorize avec un DataFrame

In [73]:
# on aurait pu utiliser from pyspark.ml.feature import CountVectorizer , IDF
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,aa,aaa,aaaaaaaa,aaaaaaaaaaaaagaacgc,aaaaaaaaaaaagttaccc,aaaaaaaaaaaexon,aaaaaaaaaaahla,aaaaaggaaatatggcaatacatccatgatgttttccagttaacataggaattaccagataaatactgttaaactcttgtccagtaacaagagtt,aaaacaattgttccattgtttaaatctcagatcctggtaaaatgttcttaatttgtctgtgtacattttcctttcatggacagaccattggagtacatta,aaaaccacttcatccaccgagacattgctgccagaaactgcctcttgacctgtccaggccctggaagagtggccaagattggagacttcgg,...,ωth,ωwhere,ωθ,ϊι,ϊν,ϊϊια,ϋϋfold,όηthespot,όο,όοο
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# LDA avec mllib.clustering

In [55]:
# Note pour plus tard :Pour le CNAM, je pense que j'y coupera pas. 

from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

In [56]:
# Cluster the documents into three topics using LDA
ldaModel = LDA.train(parts_rdd, k=3)

Py4JJavaError: An error occurred while calling o390.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 15.0 failed 1 times, most recent failure: Lost task 0.0 in stage 15.0 (TID 41, localhost, executor driver): java.lang.ClassCastException: java.lang.String cannot be cast to java.util.List
	at org.apache.spark.mllib.api.python.PythonMLLibAPI$$anonfun$7.apply(PythonMLLibAPI.scala:545)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:410)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:394)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1334)
	at scala.collection.generic.Growable$class.$plus$plus$eq(Growable.scala:59)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:104)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:48)
	at scala

# LDA avec Gensim

In [76]:
from gensim import matutils, models
import scipy.sparse

In [77]:
# One of the required inputs is a term-document matrix
tdm = data_dtm.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,...,980,981,982,983,984,985,986,987,988,989
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaaaaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaaaaaaaaaagaacgc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaaaaaaaaaaagttaccc,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [80]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [82]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=20, passes=10)
lda.print_topics()

[(0,
  '0.022*"current" + 0.020*"control" + 0.017*"voltage" + 0.009*"cell" + 0.008*"constant" + 0.007*"present" + 0.007*"output" + 0.007*"culture" + 0.006*"embodiments" + 0.006*"insulin"'),
 (1,
  '0.015*"transform" + 0.011*"example" + 0.011*"block" + 0.010*"unit" + 0.010*"prediction" + 0.008*"image" + 0.008*"signal" + 0.007*"video" + 0.006*"reference" + 0.006*"information"'),
 (2,
  '0.010*"signal" + 0.010*"data" + 0.008*"unit" + 0.008*"information" + 0.008*"example" + 0.008*"second" + 0.007*"device" + 0.007*"reference" + 0.006*"video" + 0.006*"present"'),
 (3,
  '0.018*"inhibitor" + 0.011*"pde" + 0.010*"embodiments" + 0.009*"hz" + 0.008*"administered" + 0.006*"example" + 0.005*"cases" + 0.005*"comprises" + 0.005*"instances" + 0.005*"sodium"'),
 (4,
  '0.011*"element" + 0.010*"tx" + 0.008*"device" + 0.007*"end" + 0.007*"example" + 0.006*"embodiments" + 0.006*"embodiment" + 0.005*"comprises" + 0.005*"pitch" + 0.005*"detection"'),
 (5,
  '0.016*"portion" + 0.012*"second" + 0.011*"surfac

# Nettoyage du texte PoS tags : Nouns Only

In [83]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [84]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = description['description'].apply(nouns)
data_nouns

[nltk_data] Downloading package punkt to /Users/peff/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/peff/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


0      EP3544210 application claims priority Patent A...
1      EP3546818 |BACKGROUND.|1 Field invention lamps...
2      EP3378433 |CROSS-REFERENCE TO RELATED APPLICAT...
3      EP2859171 FIELD|The disclosure drilling boreho...
4      WO2015/000987 |Method concentration substance ...
5      EP3567461 FIELD|The application field technolo...
6      EP3557422 FIELD|The application field storage ...
7      EP3541049 application claims priority Patent A...
8      WO2018/112503 STORAGE ACCESS SYSTEM|RELATED AP...
9      WO2018/109739 |TITLE| FIRE-PREVENTION CONTROL ...
10     WO2018/097632 |METHOD AND DEVICE FOR PROVIDING...
11     WO2018/138412 COMPOSITIONS WITH ENHANCED COLD ...
12     WO2018/113590 |VIDEO EXTENDERS CONTROL METHODS...
13     WO2018/115893 |Apparatus Mixing|The invention ...
14     EP3258868 |FIELD THE INVENTION|The invention f...
15     WO2016/077295 |IMPLANTABLE MEDICAL DEVICES WIT...
16     EP3086589 FIELD|The invention field communicat...
17     WO2015/089421 |DESCRIPTI

In [87]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
#add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
#                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
#stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words='english')
data_cvn = cvn.fit_transform(data_nouns)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,00,000,000001,00000allylamine,0001,00012450allylamine,000148,000262,0003,00030,...,ϕleftnominal,ϕleftrepro,ϕoffset_,ϕoffset_leftnominal,ϕoffset_leftrepro,ϕoffset_rightrepro,ϕoffset_value,ϕrepro_offset_value,ϕrightnominal,ϕrightrepro
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [92]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"mg" + 0.009*"embodiments" + 0.009*"example" + 0.008*"group" + 0.008*"acid" + 0.007*"compound" + 0.007*"composition" + 0.007*"invention" + 0.006*"ml" + 0.005*"embodiment"'),
 (1,
  '0.014*"device" + 0.012*"example" + 0.011*"data" + 0.011*"fig" + 0.008*"embodiments" + 0.008*"information" + 0.007*"embodiment" + 0.007*"portion" + 0.005*"sequence" + 0.005*"unit"')]